In [1]:
import tensorflow as tf
import numpy as np
import os, logging, pickle
import time
from utils import random_batch, normalize, similarity, loss_cal, optim
from configuration import get_config
from tensorflow.contrib import rnn

config = get_config()

Namespace(M=5, N=4, beta1=0.5, beta2=0.9, comment='', hidden=768, hop=0.01, iteration=100000, loss='softmax', lr=0.01, model_num=6, model_path='./tisv_model', nfft=512, noise_filenum=16, noise_path='/datadrive2/dalon/diarization-experiments/Speaker_Verification/data/VCTK-Corpus/noise', num_layer=3, optim='sgd', proj=256, restore=False, sr=8000, tdsv=False, tdsv_frame=80, test_path='/datadrive2/dalon/diarization-experiments/Speaker_Verification/data/VCTK-Corpus/test', tisv_frame=50, train=False, train_path='/datadrive2/dalon/diarization-experiments/Speaker_Verification/data/VCTK-Corpus/train', window=0.025)


In [2]:
from collections import defaultdict

# Data selection

Roughly in dev set:

Voxceleb 2</br >
Number of speakers = 5,994

Voxceleb 1</br >
Number of speakers = 1,211

In [3]:
# Voxceleb-1 structure
# [['/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-1/wav/id10266/SHI6yGzrwLg/00001.wav',
#   '/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-1/cleaned-data/id10266/SHI6yGzrwLg/00001.npy',
#   (12, 40, 50)],
# voxceleb1_file_list[:2]

In [4]:
# # Voxceleb1 distribution
# voxceleb1_file_path = "/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-1/voxceleb-1.b"
# voxceleb1_file_list = np.load(voxceleb1_file_path)
# voxceleb1_clean_list = [[x[1], int(x[2][0])] for x in voxceleb1_file_list if x[2][0] != 0]
# voxceleb1_size_list = np.array([x[1] for x in voxceleb1_clean_list])
# print(f'Total # of Utterances = {len(voxceleb1_size_list)}')
# print(f'Max = {voxceleb1_size_list.max()}\nMin = {voxceleb1_size_list.min()}\nMean = {voxceleb1_size_list.mean()}')

In [5]:
# # Voxceleb2 distribution
# voxceleb2_file_path = "/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-2/dev/voxceleb-2.b"
# voxceleb2_file_list = np.load(voxceleb2_file_path)
# voxceleb2_clean_list = [[x[1], int(x[2][0])] for x in voxceleb2_file_list if x[2][0] != 0]
# voxceleb2_size_list = np.array([x[1] for x in voxceleb2_clean_list])
# print(f'Total # of Utterances = {len(voxceleb2_size_list)}')
# print(f'Max = {voxceleb2_size_list.max()}\nMin = {voxceleb2_size_list.min()}\nMean = {voxceleb2_size_list.mean()}')

In [6]:
# Voxceleb1 distribution cleanup
dataset_name = 'voxceleb-2-train-v1'
voxceleb_file_path = "/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-2/dev/voxceleb-2.b"
save_dir = "/datadrive2/dalon/diarization-experiments/voxceleb-dataset/voxceleb-2/train-data-v1"
log_file = os.path.abspath(dataset_name + ".logs")
logging.basicConfig(
    filename=log_file,
    level=logging.DEBUG,
    format="%(asctime)s:%(levelname)s:%(message)s"
    )
print(f'Log path: {log_file}')


voxceleb_file_list = np.load(voxceleb_file_path)
voxceleb_map_speaker = [[x[1].split('/')[-3], x[1]] for x in voxceleb_file_list if x[2][0] != 0]

Log path: /datadrive2/dalon/diarization-experiments/Speaker_Verification/Notebooks/voxceleb-2-train-v1.logs


In [7]:
os.makedirs(save_dir)

In [8]:
logging.info(f'Saving directory: {save_dir}')
voxceleb_map_speaker = defaultdict(list)
for x in voxceleb_file_list:
    if x[2][0] != 0:
        voxceleb_map_speaker[x[1].split('/')[-3]].append(x[1])

number_of_speakers = len(voxceleb_map_speaker)
logging.info(f'Number of Speakers: {number_of_speakers}')

In [ ]:
try:
    allfiles = []
    for idx, speakerid in enumerate(voxceleb_map_speaker):
        logging.info(f'Processing: {idx + 1}/{number_of_speakers}')
        save_path = os.path.join(save_dir, speakerid + ".npy")
        utterances = []
        for files in voxceleb_map_speaker[speakerid]:
            vals = np.load(files)
            utterances.extend(vals.tolist())
        utterances = np.array(utterances)
        logging.info(f'Saving: {save_path}')
        logging.info(utterances.shape)
        np.save(save_path, utterances)
        allfiles.append([save_path, utterances.shape])

    logging.info("Completed!")    
except Exception as e:
    logging.exception(e)

In [ ]:
with open(os.path.join(save_dir, dataset_name + '.b'), "wb") as f:
    # save the distribution
    logging.info(f'Saving processed audio list to {os.path.join(save_dir, dataset_name + ".b")}')
    pickle.dump(allfiles, f)